In [2]:
pip install selenium

   ---------------------------------------- 0.0/9.4 MB ? eta -:--:--
   ------------------------ --------------- 5.8/9.4 MB 27.1 MB/s eta 0:00:01
   ---------------------------------------- 9.4/9.4 MB 25.3 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.2.3
    Uninstalling urllib3-2.2.3:
      Successfully uninstalled urllib3-2.2.3
  Attempting uninstall: typing_extensions
    Found existing installation: typing_extensions 4.11.0
    Uninstalling typing_extensions-4.11.0:
      Successfully uninstalled typing_extensions-4.11.0
  Attempting uninstall: attrs
    Found existing installation: attrs 23.1.0
    Uninstalling attrs-23.1.0:
      Successfully uninstalled attrs-23.1.0
Note: you may need to restart the kernel to use updated packages.


In [37]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time, pandas as pd, os, pickle

df=pd.read_csv("레이블셋주소")
print(df.describe())
df["id"]=df["id"].apply(lambda x:x[1:])

# 🔧 Chrome 설정
options = Options()
options.add_argument("--window-size=1920x1080")
options.add_argument("--incognito")
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)
driver.get("https://twitter.com/home")

WebDriverWait(driver, 30).until( EC.presence_of_element_located((By.XPATH, '//div[@aria-label="Timeline: Your Home Timeline"]')) )
print("✅ 로그인 완료!")

results = []
i+=1
j=0
while i<600:
    try:
        driver.get(f"https://twitter.com/i/user/{df.iloc[i]["id"]}")

        time.sleep(8)

        current_url = driver.current_url
        username = current_url.split("/")[-1]

        # 기본 정보 수집
        
        try: print(driver.find_element(By.XPATH, '//script[@data-testid="UserProfileSchema-test"]').text)
        except: print("no")
        
        try: display_name = driver.find_element(By.XPATH, '//div[@data-testid="UserName"]//span[1]').text
        except: display_name = ""

        try: bio = driver.find_element(By.XPATH, '//div[@data-testid="UserDescription"]').text
        except: bio = ""
        
        try: 
            driver.find_element(By.XPATH, '//a[contains(@href,"/verified_followers")]//span[1]/span').text
            is_private=0
        except: is_private=1
        
        try:
            script_element = driver.find_element(By.XPATH, '//script[@type="application/ld+json"]')
            json_data = script_element.get_attribute("innerHTML")
            itrtStat=json.loads(json_data)["mainEntity"]["interactionStatistic"]
        except: itrtStat=[{"interactionStatistic":"nan"}]*3

        nums_in_username = sum(c.isdigit() for c in username)
        nums_in_fullname = sum(c.isdigit() for c in display_name)
        external_url = "http" in bio
        
        results.append({
            "user_id": df.iloc[i]["id"],
            "nums/lenth username": nums_in_username/len(username),
            "fullname words": len(display_name.split()),
            "nums/lenth fullname": nums_in_fullname/len(display_name),
            "name==username": int(display_name.lower() == username.lower()),
            "profile_pic": 1,
            "description length": len(bio),
            "external URL": int(external_url),
            "private": int(is_private),
            "#posts": itrtStat[2]['userInteractionCount'],
            "#followers": itrtStat[0]['userInteractionCount'],
            "#follows": itrtStat[1]['userInteractionCount'],
            "fake": int(df.iloc[i]["label"]!="human")
        })

        print(f"✅ @{username} 크롤링 완료")
        j=0
    except Exception as e:
        print(f"❌ {df.iloc[i]["id"]} 실패: {e}")
        j+=1
        if j>2:
            i-=1
            break
    i+=1

driver.quit()

df = pd.concat([pd.read_csv("데이터셋주소"),pd.DataFrame(results)])
df.to_csv("x_user_data.csv", index=False, encoding='utf-8-sig')
print("📦 저장 완료: x_user_data.csv")
print(df.describe())

               id   label
count      260000  260000
unique     260000       2
top     u17140361   human
freq            1  130000
✅ 로그인 완료!
no
❌ 1189929383315804160 실패: division by zero

✅ @JaviPerezCampos 크롤링 완료

✅ @pikulsomesh 크롤링 완료
no
❌ 1545313657 실패: division by zero

✅ @TOITopStories 크롤링 완료

✅ @juandiegoluna 크롤링 완료

✅ @RadioHuancavilk 크롤링 완료

✅ @afzalxiraj 크롤링 완료

✅ @_pairme_ 크롤링 완료

✅ @radiio_avaa 크롤링 완료

✅ @SEMARNAT_mx 크롤링 완료

✅ @Nagaraj58722205 크롤링 완료

✅ @lexfeltham 크롤링 완료
no
❌ 100246862 실패: division by zero

✅ @fstoppers 크롤링 완료

✅ @profjkim 크롤링 완료

✅ @PnoidBill 크롤링 완료
no
❌ 271688756 실패: division by zero

✅ @lgbtmap 크롤링 완료

✅ @BottlaVoice 크롤링 완료
no
❌ 124864615 실패: division by zero

✅ @crowdtangle 크롤링 완료

✅ @GTLabels 크롤링 완료

✅ @Frankjee 크롤링 완료

✅ @Vijaynarain 크롤링 완료

✅ @feamster 크롤링 완료

✅ @moonbreeze2 크롤링 완료

✅ @LOKENDR76761639 크롤링 완료
no
❌ 2313618144 실패: division by zero

✅ @latinus_us 크롤링 완료

✅ @LukasLechnerDev 크롤링 완료

✅ @PTFindr_ 크롤링 완료

✅ @ConnectOneBank 크롤링 완료

✅ @mudasirba

In [38]:
df.value_counts()

user_id              nums/lenth username  fullname words  nums/lenth fullname  name==username  profile_pic  description length  external URL  private  #posts  #followers  #follows  fake
823057               0.000000             6               0.0                  0               1            113                 0             0        53183   2838        4635      0       1
1251485283532578816  0.166667             2               0.0                  0               1            33                  0             0        8       14          116       0       1
1319297349978451968  0.571429             1               0.0                  0               1            110                 0             0        67      8           50        0       1
1314704599786442753  0.100000             2               0.0                  0               1            34                  0             1        28491   26970       6641      0       1
1306056392319533056  0.000000             2       